<a href="https://colab.research.google.com/github/BuczynskiRafal/ML/blob/main/unsupervised/09_pca_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import bibliotek


In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

np.set_printoptions(precision=4, suppress=True, edgeitems=5, linewidth=200)

# Załadowanie danych


In [8]:
df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [9]:
# WYcinam dane i zmienną docelową - dostaję 13 wymiarów - trzeba to zredukować
data = df.iloc[:, 1:]
target = df.iloc[:, 0]
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [10]:
# ilość klas
target.value_counts()


2    71
1    59
3    48
Name: 0, dtype: int64

# Podział na zbiór treningowy i testowy


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (133, 13)
X_test shape: (45, 13)


# Standaryzacja

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_train_std[:5]

array([[ 0.0963,  1.3998, -0.025 ,  0.6499,  0.9976, -1.4896, -0.6591, -0.1947, -0.764 ,  1.7506, -1.683 , -1.809 , -0.6394],
       [ 1.4514,  0.1592,  0.4188,  0.1753, -0.6295, -1.0431, -1.3515,  0.6095, -0.5967,  1.8759, -1.4632, -1.2639, -0.3003],
       [-0.3805,  1.0874, -0.025 ,  0.6499,  0.4799, -1.01  , -0.8497, -1.5619, -1.2658, -0.0859, -0.7596, -1.8664, -0.4853],
       [ 0.6107,  1.1231, -0.6538,  0.0171, -0.8514, -1.1423, -1.5722,  1.7355, -1.1989,  0.2063, -0.6277, -1.1061, -0.5469],
       [-0.1797, -0.8762, -0.173 , -0.4574,  1.6633, -1.3242, -0.7995, -1.2402, -1.0986, -0.4616, -0.8476, -1.8664, -0.3928]])

# PCA

In [18]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
X_train_pca.shape

(133, 3)

In [19]:
# Wyjaśniona wariancja
results = pd.DataFrame(data={'explained_variance_ratio': pca.explained_variance_ratio_})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results



,explained_variance_ratio,cumulative,component
0,0.360492,0.360492,1
1,0.192976,0.553468,2
2,0.111790,0.665258,3


In [20]:
fig = go.Figure(data=[go.Bar(x=results['component'], y=results['explained_variance_ratio'], name='explained variance ratio'),
                      go.Scatter(x=results['component'], y=results['cumulative'], name='cumulative explained variance')],
                layout=go.Layout(title=f'PCA - {pca.n_components_} components', width=950, template='plotly_dark'))
fig.show()

In [21]:
X_train_pca_df = pd.DataFrame(data=np.c_[X_train_pca, y_train], columns=['pca1', 'pca2', 'pca3', 'target'])
X_train_pca_df.head()

,pca1,pca2,pca3,target
0,-2.965605,-1.895394,-0.460631,3.0
1,-2.486859,-2.018958,-0.978453,3.0
2,-2.172327,-0.307302,-0.464973,3.0
3,-3.056219,-0.192533,-1.067406,3.0
4,-1.333874,-0.195871,-1.380693,3.0


In [22]:
px.scatter_3d(X_train_pca_df, x='pca1', y='pca2', z='pca3', color='target', template='plotly_dark', width=950)


In [23]:
X_train_pca[:5]


array([[-2.9656, -1.8954, -0.4606],
       [-2.4869, -2.019 , -0.9785],
       [-2.1723, -0.3073, -0.465 ],
       [-3.0562, -0.1925, -1.0674],
       [-1.3339, -0.1959, -1.3807]])

In [24]:
X_test_pca[:5]

array([[-1.5763,  1.9496,  0.6938],
       [ 0.8388,  2.021 , -0.3328],
       [-2.2827, -1.9613,  0.6707],
       [-2.6468, -0.4431, -0.8172],
       [-0.5195,  2.336 , -0.1247]])